# Week 4 — Data Preparation for Analysis
_Transforming raw data into actionable insights_

**Demo Notebook Goals**
- Show practical, classroom-ready examples for:
  - Handling missing values
  - Duplicates & inconsistent entries
  - Type conversions (numeric, datetime, categorical, encodings)
  - Working with dates (extraction & resampling)
  - Renaming & reordering columns
  - Combining datasets (merge & concat)
  - Simple outlier detection & treatment


## Setup

In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(7)

: 

## 1) Handling Missing Values
We'll create a small DataFrame with deliberate missing values and demonstrate how to **detect**, **drop**, and **impute** them.

In [2]:

df_mv = pd.DataFrame({
    "First Score": [np.nan, 95, 88, np.nan],
    "Second Score": [75, np.nan, 82, 90],
    "Group": ["A", "A", "B", "B"]
})
df_mv

,First Score,Second Score,Group
0,NaN,75.0,A
1,95.0,NaN,A
2,88.0,82.0,B
3,NaN,90.0,B


In [3]:

# Detect missing values
print("Missing values per column: \n ")
print(df_mv.isna().sum())



Missing values per column: 
 
First Score     2
Second Score    1
Group           0
dtype: int64


In [4]:
# Drop rows with any missing values (demo)
dropped = df_mv.dropna()
print("\nRows after dropna(): \n ")
print(dropped)


Rows after dropna(): 
 
   First Score  Second Score Group
2         88.0          82.0     B


### Imputation: fill numeric columns with column mean

In [5]:
df_mv

,First Score,Second Score,Group
0,NaN,75.0,A
1,95.0,NaN,A
2,88.0,82.0,B
3,NaN,90.0,B


In [6]:

mean_imputed = df_mv.copy()
for col in ["First Score", "Second Score"]:
    mean_imputed[col] = mean_imputed[col].fillna(mean_imputed[col].mean())
mean_imputed

,First Score,Second Score,Group
0,91.5,75.000000,A
1,95.0,82.333333,A
2,88.0,82.000000,B
3,91.5,90.000000,B


## 2) Duplicates & Inconsistent Entries
We'll spot duplicates and standardize string columns (e.g., `.str.strip()`, `.str.title()`).

In [7]:

df_dup = pd.DataFrame({
    "EmployeeID": [1, 2, 3, 3],
    "Name": ["alice  ", "Bob", "Carol", "Carol"],
    "Department": ["hr", "Engineering", "SALES", "SALES"]
})
print("Original: \n")
print(df_dup)

# Detect duplicates: marks True for subsequent duplicates
print("\nDuplicated mask: \n")
print(df_dup.duplicated())

# Remove exact duplicate rows
deduped = df_dup.drop_duplicates()
print("\nAfter drop_duplicates(): \n")
print(deduped)

# Standardize text
standard = deduped.copy()
standard["Name"] = standard["Name"].str.strip().str.title()
standard["Department"] = (standard["Department"]
                          .str.strip()
                          .str.upper()
                          .replace({"HR":"HR", "ENGINEERING":"Engineering", "SALES":"Sales"}))
standard

Original: 

   EmployeeID     Name   Department
0           1  alice             hr
1           2      Bob  Engineering
2           3    Carol        SALES
3           3    Carol        SALES

Duplicated mask: 

0    False
1    False
2    False
3     True
dtype: bool

After drop_duplicates(): 

   EmployeeID     Name   Department
0           1  alice             hr
1           2      Bob  Engineering
2           3    Carol        SALES


,EmployeeID,Name,Department
0,1,Alice,HR
1,2,Bob,Engineering
2,3,Carol,Sales


## 3) Type Conversions
Common fixes include converting object/text to **numeric** and **datetime**, and turning string categories into **categorical**.

In [8]:

df_types = pd.DataFrame({
    "hire_date": ["2020-01-05", "03/14/2019", " 2018/7/1 "],
    "salary_text": ["65000", "120000", "70000"],
    "department": ["Engineering", "HR", "Sales"]
})

# Numeric
df_types["salary"] = pd.to_numeric(df_types["salary_text"], errors="coerce")

# Datetime (coerce handles messy formats)
df_types["hire_date_parsed"] = pd.to_datetime(df_types["hire_date"], errors="coerce")

# Categorical
df_types["department"] = df_types["department"].astype("category")

# Encodings
one_hot = pd.get_dummies(df_types[["department"]], drop_first=False, dtype=int)
label_codes, uniques = pd.factorize(df_types["department"])

df_types

,hire_date,salary_text,department,salary,hire_date_parsed
0,2020-01-05,65000,Engineering,65000,2020-01-05
1,03/14/2019,120000,HR,120000,NaT
2,2018/7/1,70000,Sales,70000,NaT


In [9]:
one_hot

,department_Engineering,department_HR,department_Sales
0,1,0,0
1,0,1,0
2,0,0,1


In [10]:
label_codes

array([0, 1, 2])

In [11]:
uniques

CategoricalIndex(['Engineering', 'HR', 'Sales'], categories=['Engineering', 'HR', 'Sales'], ordered=False, dtype='category')

## 4) Working with Dates: Extraction & Resampling
We'll create a small time series and demonstrate **downsampling** (daily → monthly) and **upsampling** (monthly → daily).

In [12]:

# Daily series for 60 days
dates = pd.date_range("2023-01-01", periods=60, freq="D")
sales = pd.Series(np.random.randint(50, 200, size=len(dates)), index=dates, name="sales")
daily_df = sales.to_frame()
daily_df.head()

,sales
2023-01-01,75
2023-01-02,117
2023-01-03,153
2023-01-04,142
2023-01-05,192


In [13]:

# Extract components
daily_df = daily_df.assign(
    year=daily_df.index.year,
    month=daily_df.index.month,
    day=daily_df.index.day
)
daily_df.head()

,sales,year,month,day
2023-01-01,75,2023,1,1
2023-01-02,117,2023,1,2
2023-01-03,153,2023,1,3
2023-01-04,142,2023,1,4
2023-01-05,192,2023,1,5


In [14]:

# Downsample: daily -> monthly totals, ME month end
monthly_sales = daily_df["sales"].resample("ME").sum()
print("Monthly totals:")
monthly_sales



Monthly totals:


2023-01-31    3728
2023-02-28    3522
2023-03-31     169
Freq: ME, Name: sales, dtype: int64

In [15]:

# Upsample: monthly -> daily (forward-fill, for demo) D daily frequency
daily_from_monthly = monthly_sales.resample("D").ffill()
daily_from_monthly.head(35)

2023-01-31    3728
2023-02-01    3728
2023-02-02    3728
2023-02-03    3728
2023-02-04    3728
2023-02-05    3728
2023-02-06    3728
2023-02-07    3728
2023-02-08    3728
2023-02-09    3728
2023-02-10    3728
2023-02-11    3728
2023-02-12    3728
2023-02-13    3728
2023-02-14    3728
2023-02-15    3728
2023-02-16    3728
2023-02-17    3728
2023-02-18    3728
2023-02-19    3728
2023-02-20    3728
2023-02-21    3728
2023-02-22    3728
2023-02-23    3728
2023-02-24    3728
2023-02-25    3728
2023-02-26    3728
2023-02-27    3728
2023-02-28    3522
2023-03-01    3522
2023-03-02    3522
2023-03-03    3522
2023-03-04    3522
2023-03-05    3522
2023-03-06    3522
Freq: D, Name: sales, dtype: int64

## 5) Renaming & Reordering Columns

In [16]:

demo = pd.DataFrame({
    "Name": ["Alice", "Bob"],
    "Age": [25, 31],
    "Qual": ["MSc", "BSc"],
    "H_Date": ["2020-03-15", "2018-06-01"]
})
print("Before: \n ")
print(demo)

demo = demo.rename(columns={"Qual": "Qualification", "H_Date": "Hire_Date"})
new_order = ["Name", "Hire_Date", "Age", "Qualification"]
demo = demo[new_order]
print("\nAfter: \n")
demo

Before: 
 
    Name  Age Qual      H_Date
0  Alice   25  MSc  2020-03-15
1    Bob   31  BSc  2018-06-01

After: 



,Name,Hire_Date,Age,Qualification
0,Alice,2020-03-15,25,MSc
1,Bob,2018-06-01,31,BSc


## 6) Combining Datasets: Merge & Concatenate

In [17]:

df_emps = pd.DataFrame({
    "employee": ["Bob", "Jake", "Lisa", "Sue"],
    "group": ["Accounting", "Engineering", "Engineering", "HR"]
})
df_sup = pd.DataFrame({
    "group": ["Accounting", "Engineering", "HR"],
    "supervisor": ["Carly", "Guido", "Steve"]
})

left_join = pd.merge(df_emps, df_sup, on="group", how="left")
inner_join = pd.merge(df_emps, df_sup, on="group", how="inner")
outer_join = pd.merge(df_emps, df_sup, on="group", how="outer", indicator=True)

print("Left join:\n", left_join, "\n")
print("Inner join:\n", inner_join, "\n")
print("Outer join (with indicator):\n", outer_join)

Left join:
   employee        group supervisor
0      Bob   Accounting      Carly
1     Jake  Engineering      Guido
2     Lisa  Engineering      Guido
3      Sue           HR      Steve 

Inner join:
   employee        group supervisor
0      Bob   Accounting      Carly
1     Jake  Engineering      Guido
2     Lisa  Engineering      Guido
3      Sue           HR      Steve 

Outer join (with indicator):
   employee        group supervisor _merge
0      Bob   Accounting      Carly   both
1     Jake  Engineering      Guido   both
2     Lisa  Engineering      Guido   both
3      Sue           HR      Steve   both


In [18]:

# Concatenation
top = pd.DataFrame({"A": [1,2], "B": [3,4]})
bottom = pd.DataFrame({"A": [5,6], "B": [7,8]})
vertical = pd.concat([top, bottom], axis=0, ignore_index=True)

left = pd.DataFrame({"A": [1,2,3]})
right = pd.DataFrame({"B": [10,20,30]})
horizontal = pd.concat([left, right], axis=1)

print("Vertical concat:\n", vertical, "\n")
print("Horizontal concat:\n", horizontal)

Vertical concat:
    A  B
0  1  3
1  2  4
2  5  7
3  6  8 

Horizontal concat:
    A   B
0  1  10
1  2  20
2  3  30


## 7) Handling Outliers: Z-Score & IQR

In [19]:

data = {'Salary': [50000, 52000, 56000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 800000]}
df_out = pd.DataFrame(data)

# Z-score (sample std by default)
df_out['z_score'] = (df_out['Salary'] - df_out['Salary'].mean()) / df_out['Salary'].std()

# Flag outliers by z-score > 3
outliers_z = df_out[np.abs(df_out['z_score']) > 3]
outliers_z



,Salary,z_score
10,800000,3.009658


In [20]:
df_out

,Salary,z_score
0,50000,-0.383765
1,52000,-0.374716
2,56000,-0.356618
3,60000,-0.338520
4,65000,-0.315897
5,70000,-0.293274
6,75000,-0.270651
7,80000,-0.248028
8,85000,-0.225406
9,90000,-0.202783


In [22]:
df_trimmed = df_out[np.abs(df_out['z_score'])
<= 3]
df_trimmed

,Salary,z_score
0,50000,-0.383765
1,52000,-0.374716
2,56000,-0.356618
3,60000,-0.338520
4,65000,-0.315897
5,70000,-0.293274
6,75000,-0.270651
7,80000,-0.248028
8,85000,-0.225406
9,90000,-0.202783


In [23]:

# IQR method
df = df_out.copy()
Q1, Q3 = df['Salary'].quantile(0.25), df['Salary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound, upper_bound = Q1 - 1.5*IQR, Q3 + 1.5*IQR
outliers_iqr = df[(df['Salary'] < lower_bound) | (df['Salary'] > upper_bound)]

print("Z-score outliers (|z| > 3):")
print(outliers_z)

print("\nIQR bounds:", lower_bound, upper_bound)
print("IQR outliers:")
print(outliers_iqr)


Z-score outliers (|z| > 3):
    Salary   z_score
10  800000  3.009658

IQR bounds: 21250.0 119250.0
IQR outliers:
    Salary   z_score
10  800000  3.009658


In [24]:
df

,Salary,z_score
0,50000,-0.383765
1,52000,-0.374716
2,56000,-0.356618
3,60000,-0.338520
4,65000,-0.315897
5,70000,-0.293274
6,75000,-0.270651
7,80000,-0.248028
8,85000,-0.225406
9,90000,-0.202783


In [25]:
df['Salary_capped'] = np.where(df['Salary'] > upper_bound, upper_bound, df['Salary'])


In [26]:
df

,Salary,z_score,Salary_capped
0,50000,-0.383765,50000.0
1,52000,-0.374716,52000.0
2,56000,-0.356618,56000.0
3,60000,-0.338520,60000.0
4,65000,-0.315897,65000.0
5,70000,-0.293274,70000.0
6,75000,-0.270651,75000.0
7,80000,-0.248028,80000.0
8,85000,-0.225406,85000.0
9,90000,-0.202783,90000.0


In [27]:
mean_salary = df['Salary'].mean()
df['Salary_imputed'] = np.where(df['Salary'] > upper_bound, mean_salary, df['Salary'])
df

,Salary,z_score,Salary_capped,Salary_imputed
0,50000,-0.383765,50000.0,50000.000000
1,52000,-0.374716,52000.0,52000.000000
2,56000,-0.356618,56000.0,56000.000000
3,60000,-0.338520,60000.0,60000.000000
4,65000,-0.315897,65000.0,65000.000000
5,70000,-0.293274,70000.0,70000.000000
6,75000,-0.270651,75000.0,75000.000000
7,80000,-0.248028,80000.0,80000.000000
8,85000,-0.225406,85000.0,85000.000000
9,90000,-0.202783,90000.0,90000.000000


# Key Takeaways
Garbage In, Garbage Out — the quality of input data dictates the quality of insights.
Systematic data preparation (missing values, types, merges, outliers) enables reliable analysis.